In [ ]:
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10-assembly_2.12:2.4.3 pyspark-shell'
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
sc = SparkContext()
ssc = StreamingContext(sc, 5) # Create a streaming context with batch interval of 5 seconds
directKafkaStream = KafkaUtils.createDirectStream(ssc, ["water"], {"metadata.broker.list": "kafka:9093"})
sql = SQLContext(sc)

In [ ]:
def process(kafkaRdd):
    print("No. of rows: %d" % kafkaRdd.count())
    
    kafkaValuesRdd = kafkaRdd.map(lambda x: x[1])
    dataFrame = sql.read.json(kafkaValuesRdd)
    for row in dataFrame.rdd.collect():
        print(row)

In [ ]:
directKafkaStream.foreachRDD(process)

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate